In [ ]:
import requests

def read_uniprot_fasta_url(url: str) -> str:
    response = requests.get(url, timeout=30)
    response.raise_for_status()

    lines = response.text.splitlines()
    sequence = "".join(line.strip() for line in lines if not line.startswith(">"))
    return sequence

sequence = read_uniprot_fasta_url("https://rest.uniprot.org/uniprotkb/P04637.fasta")

hydrophobic = {'A','V','I','L','M','F','Y','W'}

def score(seq):
    return sum(2 if r in hydrophobic else 1 for r in seq)

def leaderboard_consecutive(sequence, k):
    leaderboard = [""]  # start empty
    best_score = 0
    best_seq = ""

    # Step-by-step expansion (like leaderboard)
    for step in range(1, k+1):
        new_board = []
        for cand in leaderboard:
            start_idx = len(cand)
            if start_idx < len(sequence):
                # only take next consecutive character
                new_board.append(cand + sequence[start_idx])
        leaderboard = new_board

        # Update best score and motif
        for cand in leaderboard:
            sc = score(cand)
            if sc > best_score:
                best_score = sc
                best_seq = cand

    # Collect all top motifs with same best score
    top_motifs = []
    for i in range(len(sequence) - k + 1):
        subseq = sequence[i:i+k]
        if score(subseq) == best_score:
            top_motifs.append((subseq, best_score))

    return best_seq, best_score, top_motifs

# Run for k=5
best, best_sc, top_list = leaderboard_consecutive(sequence, k=5)

print("Best consecutive motif:", best)
print("Best score:", best_sc)

print("\nTop Motifs with highest score:")
for motif, sc in top_list:
    print(motif, sc)